In [1]:
import utils.DataProcessing as DP
import utils.VGG as VGG
import pickle

import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn

In [2]:
cuisines = ['Asian', 'Sushi', 'Ramen', 'Japanese', 'Steak', 'BBQ', 'Korean', 
            'Seafood', 'Indonesian', 'Western', 'Cafe', 'Tea', 'Snacks', 'Desserts', 
            'Makassar', 'Coffee', 'Italian', 'Bakmi', 'Street Food', 'Bakso', 'Chinese', 
            'Healthy Food', 'American', 'European', 'Dimsum', 'Peranakan', 'Mediterranean', 
            'Middle Eastern', 'Padang', 'Manado', 'Kalimantan', 'Satay', 'Sunda', 'Thai', 
            'Bakery', 'Burger', 'Pizza', 'Japanese BBQ', 'Teppanyaki', 'Fast Food', 'Martabak', 
            'Taiwanese', 'Jawa', 'Palembang', 'Malaysian', 'Balinese', 'Grill', 'Vietnamese', 'Batak',
            'Bubble Tea', 'Beverages', 'Betawi', 'Soto', 'Yogyakarta', 'Sumatera', 'Finger Food', 'Kebab', 
            'Singaporean', 'North Indian', 'Indian', 'Medan', 'Coffee and Tea', 'Sulawesi', 'Lombok', 'Juices', 
            'Bangka', 'Mexican', 'Ice Cream', 'British', 'Aceh']

In [3]:
IMAGE_DIR_TRAIN = "data/images/kelapa-gading"
JSON_DIR_TRAIN = "data/json/kelapa-gading-restaurants.json"
epochs = 10
use_gpu = torch.cuda.is_available()
if use_gpu: print('using cuda')

using cuda


In [4]:
use_prev_dp = True
if use_prev_dp:
    file = open('data/DP/DP-kelapa-gading', 'rb')
    dtrain_set = pickle.load(file)
else:
    dtrain_set = DP.ImageDataset(IMAGE_DIR_TRAIN, JSON_DIR_TRAIN, cuisines)

In [5]:
use_prev_model = True
if use_prev_model:
    model = torch.load('data/model/model-2019164524.pt')
else:
    model = VGG.vgg_19(num_classes)

In [6]:
train_loader = DataLoader(dtrain_set,
                        batch_size=16,
                        shuffle=True,
                         num_workers=2)
num_classes = len(cuisines)
if use_gpu: model.cuda()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
loss_fn = nn.MultiLabelSoftMarginLoss()

In [7]:
#train
flag = False
for epoch in range(epochs):
    correct = 0
    total = 0
    for data, labels in train_loader:
        if use_gpu:
            data = Variable(data.cuda())
            labels = Variable(labels.cuda())
        else: 
            data = Variable(data)
            labels = Variable(labels)
        
        outputs = model(data)
        with torch.no_grad():
            sig = torch.sigmoid(outputs).data
            if torch.isnan(sig).any():
                flag = True
                break
            output_label =  sig > 0.5
            total += len(output_label)
            for i in range(len(output_label)):
                flag = True
                for j in range(len(output_label[i])):
                    if labels[i][j].item() != output_label[i][j].item():
                        flag = False
                        break
                if flag: correct += 1
        if flag:
            print('nan occours')
            break
        #backprop
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss = loss.item()
    print("loss: ", running_loss)
    print("accuracy: ", (correct / total))
                

loss:  0.12875361740589142
accuracy:  0.0003892925997330565
nan occours
loss:  0.11233311891555786
accuracy:  0.00015967153284671533


RuntimeError: CUDA out of memory. Tried to allocate 24.50 MiB (GPU 0; 5.94 GiB total capacity; 5.39 GiB already allocated; 5.31 MiB free; 8.92 MiB cached)

In [8]:
import datetime
dt = datetime.datetime.now()
str_dt = str(dt.year) + str(dt.month) + str(dt.day) + str(dt.hour) + str(dt.minute) + str(dt.second)
torch.save(model, 'data/model/model-'+str_dt+'.pt')

In [3]:
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_img = Image.open('test.jpg')
test_img = test_img.resize((224, 224))
test_arr = np.asarray(test_img)
test_data = transform(test_arr)
test_data = test_data.unsqueeze(0)
test_data = Variable(test_data.cuda())
test_output = model(test_data)
output_label = torch.sigmoid(test_output).data > 0.5
print(output_label)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0', dtype=torch.uint8)


In [8]:
torch.max(torch.sigmoid(test_output).data)

tensor(0.1666, device='cuda:0')

In [14]:
torch.isnan(torch.sigmoid(test_output).data)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0', dtype=torch.uint8)

In [23]:
a = torch.tensor([0,np.nan])
if torch.isnan(a).any(): print("test")

test


In [19]:
torch.sigmoid(test_output)

tensor([[1.1634e-01, 6.6586e-02, 6.9334e-03, 7.8153e-02, 9.9148e-04, 1.8626e-03,
         1.9262e-02, 1.6836e-02, 4.4938e-01, 1.3323e-01, 9.5941e-02, 1.4395e-02,
         4.2249e-05, 5.9750e-02, 3.3005e-04, 7.3822e-02, 6.0955e-02, 2.6179e-02,
         1.4943e-02, 7.3163e-04, 7.0844e-02, 2.5340e-06, 3.3176e-04, 7.2280e-04,
         2.2138e-03, 9.0703e-05, 1.3196e-06, 7.0321e-06, 3.7807e-05, 2.9657e-04,
         1.1235e-06, 1.8779e-03, 2.2009e-05, 7.8723e-04, 6.9233e-03, 5.5289e-06,
         3.8529e-04, 2.1933e-06, 4.1771e-07, 3.7845e-04, 1.1465e-05, 5.4919e-05,
         4.1576e-03, 4.7877e-06, 3.3316e-05, 2.9362e-06, 2.0002e-06, 2.2013e-05,
         8.4519e-07, 2.1632e-05, 6.0465e-05, 7.5168e-05, 4.3475e-05, 2.3597e-06,
         5.7954e-06, 2.2029e-06, 1.1221e-06, 6.0369e-06, 7.9962e-06, 1.2869e-06,
         9.5057e-06, 2.0194e-06, 1.9534e-06, 2.6824e-06, 5.6084e-06, 8.6618e-07,
         4.7790e-07, 3.7185e-04, 4.4265e-06, 2.9679e-06]], device='cuda:0',
       grad_fn=<SigmoidBackward>)

In [26]:
output_label

tensor([[1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
         1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0', dtype=torch.uint8)